# --- Day 13: Shuttle Search ---

https://adventofcode.com/2020/day/13

In [1]:
path = '../inputs/'

## Part 1

In [2]:
def part_1(filename):
    with open (path + filename) as file:
        lines = file.readlines() 

    arrival_time = int(lines[0].strip())
    bus_schedule = lines[1].strip()
    buses = [int(n) for n in bus_schedule.split(',') if not n == 'x']
    
    wait_times = [b - (arrival_time % b)  for b in buses]
    
    min_wait_time = min(wait_times)
    bus_with_min_wait_time = buses[wait_times.index(min_wait_time)]
    
    return min_wait_time * bus_with_min_wait_time

In [3]:
part_1('example_bus_notes.txt') # Should return 295

295

In [4]:
part_1('bus_notes.txt')

3997

## Part 2

### Brute Force

I don't even know why I tried...

This works on the small example, but takes FOREVER on the actual puzzle data.

In [5]:
# def part_2(filename, start):
#     with open (path + filename) as file:
#         lines = file.readlines()

#     bus_schedule = lines[1].strip()
#     buses = [n for n in bus_schedule.split(',')]

#     bus_ints = [int(n) for n in buses if not n == 'x']
#     max_bus = str(max(bus_ints))
#     max_bus_index = buses.index(str(max_bus))

#     offsets = [i - max_bus_index for i, b in enumerate(buses) if not b == 'x']

#     i = start + int(max_bus) - (start % int(max_bus))

#     while True:
#         test = []
        
#         for bus, offset in zip(bus_ints, offsets):
#             test.append((i + offset) % bus == 0)
        
#         if all(test) == True:
#             break
    
#         i += int(max_bus)
        
#     return i + min(offsets)

In [6]:
# part_2('example_bus_notes.txt', 1000000) # Should return 1068781

In [7]:
# I stopped this because it was running way too long...
# part_2('bus_notes.txt', 100000000000000)

### Chinese Remainder Theorem

This is what we need!   

Sources:   
https://en.wikipedia.org/wiki/Chinese_remainder_theorem   
https://www.youtube.com/watch?v=zIFehsBHB8o   
https://www.youtube.com/watch?v=ru7mWZJlRQg   

Here is the set up for 3 modulo equations:   
$
x\equiv b_1 (mod\ n_1) \newline
x\equiv b_2 (mod\ n_2) \newline
x\equiv b_3 (mod\ n_3)
$

We are interested in solving for x.   

In the puzzle, $b_i$ is the offsets between the bus times, and $n_i$ is the bus number, or the *divisor*.

It is possible to solve this system if (and only if -- $iff$) $n_i$ are *coprimes*. That is, there are no common factors among the $n_i$. (All the bus route numbers are primes, so we're good on this front.)

To solve the system, we first will need to calculate $N$, which is the product of all the $n_i$:

$N = \prod_{i=1}^{3}\ n_i$

We then need to calculate $N_i$, which is:   
$N_i = {{N}\over{n_i}}$

And one more piece that we need is $x_i$, which is the "inverse of $N_i$".   
So for each equation, we need to solve for $x_i$ in this equation:   

$N_ix_i\equiv 1 (mod\ n_i)$

Finally, putting together all the pieces, we calculate the sum of the products $b_iN_ix_i$ and then calculate the modulo $N$:   

$x = \sum_{i=1}^{3}b_iN_ix_i (mod\ N)$

In [8]:
def part_2(filename):
    with open (path + filename) as file:
        lines = file.readlines()

    bus_schedule = lines[1].strip()
    buses = [n for n in bus_schedule.split(',')]

    # bus_ints are the divisors (n_i) and offsets are the remainders (b_i)
    bus_ints = [int(n) for n in buses if not n == 'x']
    offsets = [i for i, b in enumerate(buses) if not b == 'x']
    remainders = [b - o for b, o in zip(bus_ints, offsets)]

    N = 1
    for n in bus_ints:
        N *= n
    
    N_i = [int(N/n) for n in bus_ints]
    
    print(bus_ints, remainders)
    print(N, N_i)

    x_i = [] 
    for n, x in zip(N_i, bus_ints):
        i = 1
        while True:
            if int(n * i) % x == 1:
                x_i.append(i)
                break
            i += 1
        
    product = [b*n*x for b, n, x in zip(remainders, N_i, x_i)]
    
    x = sum(product) % N
    
    return x

In [9]:
part_2('example_bus_notes.txt') # Should return 1068781

[7, 13, 59, 31, 19] [7, 12, 55, 25, 12]
3162341 [451763, 243257, 53599, 102011, 166439]


1068781

In [10]:
part_2('example2_bus_notes.txt') # Should return 3417

[17, 13, 19] [17, 11, 16]
4199 [247, 323, 221]


3417

In [11]:
part_2('example3_bus_notes.txt') # Should return 754018

[67, 7, 59, 61] [67, 6, 57, 58]
1687931 [25193, 241133, 28609, 27671]


754018

In [12]:
part_2('example4_bus_notes.txt') # Should return 779210

[67, 7, 59, 61] [67, 5, 56, 57]
1687931 [25193, 241133, 28609, 27671]


779210

In [13]:
part_2('example5_bus_notes.txt') # Should return 1261476

[67, 7, 59, 61] [67, 6, 56, 57]
1687931 [25193, 241133, 28609, 27671]


1261476

In [14]:
part_2('example6_bus_notes.txt') # Should return 1202161486

[1789, 37, 47, 1889] [1789, 36, 45, 1886]
5876813119 [3284971, 158832787, 125038577, 3111071]


1202161486

In [15]:
part_2('bus_notes.txt')

[19, 41, 37, 787, 13, 23, 29, 571, 17] [19, 32, 24, 768, -19, -19, -19, 521, -50]
1909273434898297 [100488075520963, 46567644753617, 51601984726981, 2426014529731, 146867187299869, 83011888473839, 65837014996493, 3343736313307, 112310202052841]


500033211739354